In [1]:
from sklearn import preprocessing
import pandas as pd
import numpy as np
from math import pi, sin, cos
from datetime import datetime
from holidays_es import get_provinces, Province

#from keras.models import Sequential
#from keras.layers import Dense, Activation, Dropout
#from keras.optimizers import Adam
#from keras.callbacks import EarlyStopping

## Importing data

In [ ]:
df = pd.read_json("../raw_data/202106.json", encoding_errors="ignore", lines=True)
weather = pd.read_csv("../raw_data/weather/weather_cleaned.csv")
df.head()

## Adding columns

In [ ]:
df["time"] = "time"
df["date"] = "date"
for i in range(len(df['_id'])):
    item=df['_id'][i]
    groups= item.split("T")
    date = groups[0]
    raw_time = groups[1].split(":")
    time = raw_time[0]
    df["time"][i] = time
    df["date"][i] = date

In [ ]:
df

In [ ]:
df

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')


In [ ]:
df

In [ ]:
holidays=[]
for year in [2018,2019,2020,2021]:
    temp = Province(name="madrid", year=year).holidays()
    for day in range(len(temp['national_holidays'])):
        holidays.append(temp['national_holidays'][day])
    for day in range(len(temp['regional_holidays'])):
        holidays.append(temp['regional_holidays'][day])
    for day in range(len(temp['local_holidays'])):
        holidays.append(temp['local_holidays'][day])

In [ ]:
df_holidays=pd.DataFrame(holidays, columns=['date'])
df_holidays['date'] = pd.to_datetime(df_holidays['date'], format='%Y-%m-%d')
list_holidays=list(df_holidays['date'])
df['holidays']=df['date'].isin(list_holidays)

In [ ]:
df

# THIS TAKES A LOT TO RUN

In [ ]:
len(df["stations"][172])

In [ ]:
temp=pd.DataFrame()
for i in range(len(df)):
        new_dict= df['stations'][i][42]
        new_dict['id']=df['_id'][i]
        new_dict['time']=df['time'][i]
        new_dict['date']=df['date'][i]
        new_dict['holidays']=df['holidays'][i]
        temp = temp.append(new_dict, ignore_index=True)


In [ ]:
temp

In [ ]:
temp["datetime"] = temp["date"].astype(str)+" "+temp["time"].astype(str)
tempdf['time'] = df.apply(lambda x:datetime.strptime(x['time'], '%H:%M').hour, axis=1)


In [ ]:
weather

In [ ]:
temp = temp.merge(weather, on="datetime")
temp

In [ ]:

temp['weekday'] = temp.apply(lambda x: x['date'].weekday(), axis=1)
temp['year'] = temp.apply(lambda x:x['date'].year, axis=1)
temp['month'] = temp.apply(lambda x: x['date'].month, axis=1)

In [ ]:
temp.head(2)

### Encoding circular time
temp['hour_sin'] = temp.apply(lambda x: sin(x['hour'] / 24.0 * 2 * pi), axis=1)
temp['hour_cos'] = temp.apply(lambda x: cos(x['hour'] / 24.0 * 2 * pi), axis=1)
temp['weekday_sin'] = temp.apply(lambda x: sin(x['weekday'] / 7.0 * 2 * pi), axis=1)
temp['weekday_cos'] = temp.apply(lambda x: cos(x['weekday'] / 7.0 * 2 * pi), axis=1)
temp['month_sin'] = temp.apply(lambda x: sin(((x['month'] - 5) % 12) / 12.0 * 2 * pi), axis=1)
temp['month_cos'] = temp.apply(lambda x:cos(((x['month'] - 5) % 12) / 12.0 * 2 * pi), axis=1)
temp['season_sin'] = temp.apply(lambda x: sin(((x['season'] - 3) % 4) / 4.0 * 2 * pi), axis=1)
temp['season_cos'] = temp.apply(lambda x: cos(((x['season'] - 3) % 4) / 4.0 * 2 * pi), axis=1)

In [ ]:
temp['hour_sin'] = temp.apply(lambda x: sin(x['time'] / 24.0 * 2 * pi), axis=1)
temp['hour_cos'] = temp.apply(lambda x: cos(x['time'] / 24.0 * 2 * pi), axis=1)
temp['weekday_sin'] = temp.apply(lambda x: sin(x['weekday'] / 7.0 * 2 * pi), axis=1)
temp['weekday_cos'] = temp.apply(lambda x: cos(x['weekday'] / 7.0 * 2 * pi), axis=1)
temp['month_sin'] = temp.apply(lambda x: sin(((x['month']- 5) % 12) / 12.0 * 2 * pi), axis=1)
temp['month_cos'] = temp.apply(lambda x:cos(((x['month'] - 5 ) % 12) / 12.0 * 2 * pi), axis=1)

In [ ]:
temp.month

In [ ]:
temp.drop(columns=["name","longitude","address","month","time","weekday"])
temp.head(3)

In [ ]:
#temp.to_csv("filename.processed.csv")

In [2]:

df = pd.read_json( "../raw_data/202106.json",encoding_errors='ignore', lines=True)
#weather = pd.read_csv("../raw_data/weather/weather_cleaned.csv")



In [3]:




df["time"] = "time"
df["date"] = "date"
for i in range(len(df['_id'])):
    item=df['_id'][i]
    groups= item.split("T")
    date = groups[0]
    raw_time = groups[1].split(":")
    time = raw_time[0]
    df["time"][i] = time
    df["date"][i] = date

df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')



In [4]:
holidays=[]
for year in [2018,2019,2020,2021]:
    temp = Province(name="madrid", year=year).holidays()
    for day in range(len(temp['national_holidays'])):
        holidays.append(temp['national_holidays'][day])
    for day in range(len(temp['regional_holidays'])):
        holidays.append(temp['regional_holidays'][day])
    for day in range(len(temp['local_holidays'])):
        holidays.append(temp['local_holidays'][day])


df_holidays=pd.DataFrame(holidays, columns=['date'])
df_holidays['date'] = pd.to_datetime(df_holidays['date'], format='%Y-%m-%d')
list_holidays=list(df_holidays['date'])
df['holidays']=df['date'].isin(list_holidays)



In [5]:
df


,_id,stations,time,date,holidays
0,2021-06-01T00:43:39.688214,"[{'activate': 1, 'name': 'Puerta del Sol A', '...",00,2021-06-01,False
1,2021-06-01T01:43:41.872092,"[{'activate': 1, 'name': 'Puerta del Sol A', '...",01,2021-06-01,False
2,2021-06-01T02:43:42.969250,"[{'activate': 1, 'name': 'Puerta del Sol A', '...",02,2021-06-01,False
3,2021-06-01T03:43:44.600717,"[{'activate': 1, 'name': 'Puerta del Sol A', '...",03,2021-06-01,False
4,2021-06-01T04:43:45.629938,"[{'activate': 1, 'name': 'Puerta del Sol A', '...",04,2021-06-01,False
...,...,...,...,...,...
707,2021-06-30T19:03:28.471971,"[{'activate': 1, 'name': 'Puerta del Sol A', '...",19,2021-06-30,False
708,2021-06-30T20:03:28.506666,"[{'activate': 1, 'name': 'Puerta del Sol A', '...",20,2021-06-30,False
709,2021-06-30T21:03:29.744034,"[{'activate': 1, 'name': 'Puerta del Sol A', '...",21,2021-06-30,False
710,2021-06-30T22:03:31.966720,"[{'activate': 1, 'name': 'Puerta del Sol A', '...",22,2021-06-30,False


In [6]:
temp=pd.DataFrame()
#for final whole model
# for i in range(len(df)):
#     for j in range(len(df["stations"][1])):
#         new_dict= df['stations'][i][0]
#         new_dict['id']=df['_id'][i]
#         new_dict['time']=df['time'][i]
#         new_dict['date']=df['date'][i]
#         new_dict['holidays']=df['holidays'][i]
#         temp = temp.append(new_dict, ignore_index=True)


for i in range(len(df)):
    try:
        new_dict= df['stations'][i][station]
        new_dict['id']=df['_id'][i]
        new_dict['time']=df['time'][i]
        new_dict['date']=df['date'][i]
        new_dict['holidays']=df['holidays'][i]
        temp = temp.append(new_dict, ignore_index=True)
    except:
        quit()

temp
temp["datetime"] = temp["date"].astype(str)+" "+temp["time"].astype(str)


""


In [ ]:
temp

In [1]:
import pandas as pd
weather = pd.read_csv("../raw_data/weather/weather_cleaned.csv")
weather

,feels_like,weather_main,datetime
0,3.08,Clear,2018-01-01 00
1,2.88,Clear,2018-01-01 01
2,2.11,Clear,2018-01-01 02
3,2.48,Clear,2018-01-01 03
4,1.69,Clear,2018-01-01 04
...,...,...,...
41488,28.51,Clear,2022-08-27 19
41489,27.64,Clear,2022-08-27 20
41490,27.38,Clear,2022-08-27 21
41491,26.65,Clear,2022-08-27 22


In [2]:
temp = temp.merge(weather, on="datetime")




temp['weekday'] = temp.apply(lambda x: x['date'].weekday(), axis=1)
temp['year'] = temp.apply(lambda x:x['date'].year, axis=1)
temp['month'] = temp.apply(lambda x: x['date'].month, axis=1)


temp['hour_sin'] = temp.apply(lambda x: sin(x['time'] / 24.0 * 2 * pi), axis=1)
temp['hour_cos'] = temp.apply(lambda x: cos(x['time'] / 24.0 * 2 * pi), axis=1)
temp['weekday_sin'] = temp.apply(lambda x: sin(x['weekday'] / 7.0 * 2 * pi), axis=1)
temp['weekday_cos'] = temp.apply(lambda x: cos(x['weekday'] / 7.0 * 2 * pi), axis=1)
temp['month_sin'] = temp.apply(lambda x: sin(((x['month'] - 5) % 12) / 12.0 * 2 * pi), axis=1)
temp['month_cos'] = temp.apply(lambda x:cos(((x['month'] - 5) % 12) / 12.0 * 2 * pi), axis=1)

temp.drop(columns=["name","longitude","address","month","time","weekday"])

temp.to_csv(basename+"_processed.csv", index=False)


0        2018-01-01 00
1        2018-01-01 01
2        2018-01-01 02
3        2018-01-01 03
4        2018-01-01 04
             ...      
41488    2022-08-27 19
41489    2022-08-27 20
41490    2022-08-27 21
41491    2022-08-27 22
41492    2022-08-27 23
Name: datetime, Length: 41493, dtype: object